In [3]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import sqlalchemy as sal
import psycopg2
import arcgis
from arcgis.gis import GIS
import sys
sys.path.append('../')
from gis_resources import san_diego_county_zips
import os
from utils import get_config

In [2]:
from arcgis.features import GeoAccessor

In [4]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=get_config("arcgis","clientid"))

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=hiJa7OZ02iA0P1EdQ0mRA6kXbqOOw2&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
# df = pd.DataFrame(
#     {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
#      'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
#      'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
#      'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]})

In [5]:
# gdf = geopandas.GeoDataFrame(
#     df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

In [6]:
# print(gdf.head())

In [7]:
# world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# # We restrict to South America.
# ax = world[world.continent == 'South America'].plot(
#     color='white', edgecolor='black')

# # We can now plot our ``GeoDataFrame``.
# gdf.plot(ax=ax, color='red')

# plt.show()

In [8]:
# df

In [5]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")

In [6]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [7]:
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select * from ca_business')

        # display the PostgreSQL database server version
ca_business_result = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

In [9]:
ca_business_result_list = [(row[1],row[4],row[5],row[7],row[8],row[9]) for row in ca_business_result]
#lat_long_list[:10]

In [10]:
# Create a DataFrame
# There could be some limitation to keeping `categories` as column since arcgis was messing up the data
# , could it be a keyword ? Let's just change the name to 'business_type'
dataFrame = pd.DataFrame(ca_business_result_list,
              columns=("business_name","latitude", "longitude","zip", "business_type","city")
              );

In [11]:
dataFrame.head(2)

,business_name,latitude,longitude,zip,business_type,city
0,Alley 6 Craft Distillery,38.6281104,-122.8759307,95448.0,[Distillery],Healdsburg
1,Amami Sushi Bistro,37.6201856,-122.410203,94066.0,[Sushi restaurant],San Bruno


In [12]:
# since some of the zip are not present in ca_business table
dataFrame = dataFrame[~dataFrame['zip'].isna()]

In [ ]:
dataFrame = dataFrame.astype({"zip": int})

In [14]:
# Pick the data for only San Diego County
san_diego_county_zips_lst = san_diego_county_zips()
san_diego_county_zips_lst[:10]

[91910, 91911, 91913, 91914, 91915, 91916, 91917, 91931, 91932, 91934]

In [15]:
dataFrame = dataFrame[dataFrame["zip"].isin(san_diego_county_zips_lst)]
dataFrame.head(2)

,business_name,latitude,longitude,zip,business_type,city
4,Internet Solutions For Less,33.1863539,-117.25029649999999,92081,"[Website designer, Design agency, Internet mar...",Vista
23,Wetzel's Pretzels,32.708565199999995,-117.17027739999999,92101,[Pretzel store],San Diego


In [16]:
# Converting the dataframe with lat/long column to geo dataframe using geopandas
gdf = geopandas.GeoDataFrame(dataFrame, geometry=geopandas.points_from_xy(dataFrame.longitude, dataFrame.latitude))

In [17]:
ca_business_sgdf = GeoAccessor.from_geodataframe(gdf, inplace=False, column_name='SHAPE')

In [19]:
#ca_business_sgdf.head(5)
ca_business_sgdf['business_type'].fillna('',inplace=True)
ca_business_sgdf['business_type']=ca_business_sgdf.business_type.apply(lambda x: ','.join(x))


In [20]:
ca_business_sgdf=ca_business_sgdf.drop(['latitude','longitude'], axis=1)

In [21]:
ca_business_sgdf['zip'] = ca_business_sgdf['zip'].astype(str)

In [22]:
ca_business_sgdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46516 entries, 4 to 515948
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   business_name  46516 non-null  object  
 1   zip            46516 non-null  object  
 2   business_type  46516 non-null  object  
 3   city           46381 non-null  object  
 4   SHAPE          46516 non-null  geometry
dtypes: geometry(1), object(4)
memory usage: 2.1+ MB


In [23]:
%%time
# Convert back from a SEDF into a feature layer, and publishing on AGOL
my_new_featurelayer = ca_business_sgdf.spatial.to_featurelayer(title="San Diego County Businesses", 
                                                         gis=gis, 
                                                         folder='nourish_gis',
                                                         tags=['SanDiegoCountyBusinesses'])

CPU times: user 37 s, sys: 1.82 s, total: 38.8 s
Wall time: 1min 26s


In [28]:
pd.set_option('display.max_colwidth', None)
